In [1]:
%%capture
!pip install transformers
!pip install accelerate
!pip install bitsandbytes
!pip install librosa
!pip install faster-whisper

In [5]:
!gpustat

user                 Wed Oct 25 11:33:52 2023  470.82.01
[0] Quadro RTX 4000  | 31'C,   0 % |  7530 /  7979 MB | elidandi_2211ai08(7331M) gdm(39M) user(83M) user(55M)


In [4]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch
from transformers import BitsAndBytesConfig

model = "alexsherstinsky/Mistral-7B-v0.1-sharded" #"mistralai/Mistral-7B-v0.1" # "TinyPixel/Llama-2-7B-bf16-sharded"
bnb_config_4bit = BitsAndBytesConfig = BitsAndBytesConfig(
                                                            load_in_4bit=True,
                                                            load_in_8bit=False,
                                                            llm_int8_threshold=6.0,
                                                            llm_int8_has_fp16_weight=False,
                                                            bnb_4bit_compute_dtype="float16",
                                                            bnb_4bit_use_double_quant=True,
                                                            bnb_4bit_quant_type="nf4",
                                                        )
tokenizer = AutoTokenizer.from_pretrained(model)
model = AutoModelForCausalLM.from_pretrained( pretrained_model_name_or_path= model,
                                            device_map="auto",
                                            torch_dtype=torch.float16,
                                            offload_folder="offload",
                                            trust_remote_code=True,
                                            low_cpu_mem_usage=True,
                                            quantization_config=bnb_config_4bit)

False

===================================BUG REPORT===================================
The following directories listed in your path were found to be non-existent: {PosixPath('vs/workbench/api/node/extensionHostProcess')}
The following directories listed in your path were found to be non-existent: {PosixPath('//matplotlib_inline.backend_inline'), PosixPath('module')}
CUDA_SETUP: WARNING! libcudart.so not found in any environmental path. Searching in backup paths...
The following directories listed in your path were found to be non-existent: {PosixPath('/usr/local/cuda/lib64')}
DEBUG: Possible options found for libcudart.so: set()
CUDA SETUP: PyTorch settings found: CUDA_VERSION=113, Highest Compute Capability: 7.5.
CUDA SETUP: To manually override the PyTorch CUDA version please see:https://github.com/TimDettmers/bitsandbytes/blob/main/how_to_use_nonpytorch_cuda.md
CUDA SETUP: Required library version not found: libbitsandbytes_cuda113.so. Maybe you need to compile it from source?
CUD

/DATA/elidandi_2211ai08/anaconda3/envs/rk/lib/python3.9/site-packages/bitsandbytes/cuda_setup/main.py:166: UserWarning: Welcome to bitsandbytes. For bug reports, please run

python -m bitsandbytes


  warn(msg)
/DATA/elidandi_2211ai08/anaconda3/envs/rk/lib/python3.9/site-packages/bitsandbytes/cuda_setup/main.py:166: UserWarning: /DATA/elidandi_2211ai08/anaconda3/envs/rk did not contain ['libcudart.so', 'libcudart.so.11.0', 'libcudart.so.12.0'] as expected! Searching further paths...
  warn(msg)


RuntimeError: Failed to import transformers.integrations.bitsandbytes because of the following error (look up to see its traceback):

        CUDA Setup failed despite GPU being available. Please run the following command to get more information:

        python -m bitsandbytes

        Inspect the output of the command and see if you can locate CUDA libraries. You might need to add them
        to your LD_LIBRARY_PATH. If you suspect a bug, please take the information from python -m bitsandbytes
        and open an issue at: https://github.com/TimDettmers/bitsandbytes/issues

In [ ]:

prompt = """Give me names of top 10 actors in the world"""

In [ ]:
# tokenizer.pad_token = "[PAD]"
# tokenizer.padding_side = "left"
tokens = tokenizer(
                  prompt,
                   return_tensors='pt').input_ids.to('cuda')

In [ ]:
len(tokens[0])

1200

In [ ]:
with torch.no_grad():
    output = model.generate(tokens,
                            max_new_tokens = 120,
                            top_k = 5,
                            eos_token_id = tokenizer.eos_token_id ,
                            do_sample = True,
                            temperature = 0.1
                            )
torch.cuda.empty_cache()

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


In [ ]:
print(tokenizer.decode(output[0][len(tokens[0]):],skip_special_tokens=True))

1: 0-100
Topic 2: 101-200
Topic 3: 201-300
Topic 4: 301-400
Topic 5: 401-500
Topic 6: 501-600
Topic 7: 601-700
Topic 8: 701-800
Topic 9: 801-900
Topic 10: 
